### Install requirements

In [ ]:
%pip install -r requirements.txt

### Load Libraries, Set Global Variables and Settings

In [1]:
from __future__ import print_function
import logging
import sys
import os
import pandas as pd
import glob
from dotenv import load_dotenv
import logging
import sys
import duckdb
from googleFunctions import Form
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import ( Settings, VectorStoreIndex, SimpleDirectoryReader)
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import (StorageContext, ServiceContext)
from llama_index.core.node_parser import MarkdownNodeParser
import chromadb
import datetime
import numpy as np
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter
from sqlalchemy import *
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import QueryEngineTool
from llama_index.core.tools import ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core import SQLDatabase, Document
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.query_engine import SQLJoinQueryEngine
from llama_index.vector_stores.duckdb import DuckDBVectorStore
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores import MetadataInfo, VectorStoreInfo
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser
from sqlalchemy import (create_engine,MetaData,Table,Column,String,Integer,select,column,)
from sqlalchemy.dialects.postgresql import (INTEGER, FLOAT, BIGINT, VARCHAR, DOUBLE_PRECISION)
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.indices.struct_store import SQLTableRetrieverQueryEngine
from llama_index.core.objects import (SQLTableNodeMapping,ObjectIndex,SQLTableSchema,)
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import SQLAutoVectorQueryEngine


# NOTE: This is ONLY necessary in jupyter notebook.
import nest_asyncio

nest_asyncio.apply()

logging.getLogger().setLevel(logging.ERROR)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

Settings.callback_manager = callback_manager

load_dotenv('../Credentials/.env')

#Azure OpenAI Creds
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
credential = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_api_version = "2024-04-01-preview"
azure_openai_embedding_deployment = "text-embedding-ada-002"
embedding_model_name = "text-embedding-ada-002"
llm_model_name = "gpt-35-turbo-16k"
api_type = "azure"

#Google Creds
# CREDENTIALS_FILE = '../Credentials/credentials.json'
# CLIENT_SECRETS_FILE = '../Credentials/client_secret.json'
# SCOPES = ['https://www.googleapis.com/auth/forms.body']
# DISCOVERY_DOC = "https://forms.googleapis.com/$discovery/rest?version=v1"

user_path = "/Users/sam/dev/spaghetti_dev/"

llm = AzureOpenAI(
            model = llm_model_name,
            deployment_name = llm_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type
        )

embed_model = AzureOpenAIEmbedding(
            model = embedding_model_name,
            deployment_name = embedding_model_name,
            api_key = credential,
            azure_endpoint = endpoint,
            api_version = azure_openai_api_version,
            api_type = api_type,
            embed_batch_size=50
        )

Settings.llm = llm
Settings.embed_model = embed_model

/Users/sam/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
%pip install --upgrade duckdb

### Connect / Load DuckDB, Define SQL Query Engine

In [2]:
engine = create_engine(''.join(["duckdb:////Users/sam/dev/spaghetti_dev/MedEd-AI/Data/meded_ai_dev.duckdb"]), future=True)

metadata_obj = MetaData()

alter_schema = Table('svr_course_quiz_view', 
               metadata_obj, 
               Column("quiz_id", INTEGER), 
               Column('quiz_type', VARCHAR), 
               Column('quiz_title', VARCHAR),
               Column('history_id', BIGINT),
               Column('submission_id', BIGINT),
               Column('student_score', DOUBLE_PRECISION),
                Column('quiz_question_count', BIGINT),
               Column('quiz_points_possible', DOUBLE_PRECISION),
               Column('question_points_possible', DOUBLE_PRECISION),
               Column('answer_points_scored', DOUBLE_PRECISION),
               Column('attempt', BIGINT),
               Column('question_name',VARCHAR),
               Column('question_type', VARCHAR),
               Column('question_text', VARCHAR),
               Column('question_answer', VARCHAR),
               Column('student_answer', VARCHAR),
               autoload_with=engine, 
               extend_existing=True)

with engine.connect() as connection:
    with connection:
        metadata=MetaData()
        my_table=Table("svr_course_quiz_view", metadata, autoload_with=connection)

sql_database = SQLDatabase(engine, include_tables=["svr_course_quiz_view"])

table_node_mapping = SQLTableNodeMapping(sql_database)

table_schema = [SQLTableSchema(table_name='svr_course_quiz_view')]

obj_index = ObjectIndex.from_objects(
    table_schema,
    table_node_mapping,
    VectorStoreIndex,
)

sql_query_engine = SQLTableRetrieverQueryEngine(
    sql_database,
    obj_index.as_retriever(similarity_top_k=1),
)


/Users/sam/Library/Python/3.9/lib/python/site-packages/duckdb_engine/__init__.py:580: SAWarning: Did not recognize type 'int4' of column 'quiz_id'
  columns = self._get_columns_info(rows, domains, enums, schema)  # type: ignore[attr-defined]
/Users/sam/Library/Python/3.9/lib/python/site-packages/duckdb_engine/__init__.py:580: SAWarning: Did not recognize type 'varchar' of column 'quiz_type'
  columns = self._get_columns_info(rows, domains, enums, schema)  # type: ignore[attr-defined]
/Users/sam/Library/Python/3.9/lib/python/site-packages/duckdb_engine/__init__.py:580: SAWarning: Did not recognize type 'varchar' of column 'quiz_title'
  columns = self._get_columns_info(rows, domains, enums, schema)  # type: ignore[attr-defined]
/Users/sam/Library/Python/3.9/lib/python/site-packages/duckdb_engine/__init__.py:580: SAWarning: Did not recognize type 'int8' of column 'history_id'
  columns = self._get_columns_info(rows, domains, enums, schema)  # type: ignore[attr-defined]
/Users/sam/Library

**********
Trace: index_construction
    |_embedding -> 1.0104 seconds
**********


### Test the SQL Query Engine

In [3]:
response = sql_query_engine.query("what is the quiz_id for CBB 1 Consolidation Assessment Exercise?")
print(response)

**********
Trace: query
    |_retrieve -> 0.125854 seconds
      |_embedding -> 0.12308 seconds
    |_templating -> 1.5e-05 seconds
    |_llm -> 0.898151 seconds
    |_synthesize -> 0.926162 seconds
      |_templating -> 9e-06 seconds
      |_llm -> 0.897858 seconds
**********
The quiz_id for CBB 1 Consolidation Assessment Exercise is 277248.


### Connect to ChromaDB, Create Collection

In [5]:
# remote_db = chromadb.HttpClient(host='localhost',port=8000)
# remote_db.delete_collection("autovector")
# chroma_collection = remote_db.get_or_create_collection("autovector")

### Load PDF Course Files, Add Metadata, Create Vector Index

In [12]:
logging.getLogger().setLevel(logging.WARNING)
pd.set_option('future.no_silent_downcasting', True)
# embed that content, with metadata for where they came from/what consolidation exercise they're a part of 

reader = SimpleDirectoryReader("".join([user_path,"MedEd-AI/Data/studentguides/"]), recursive=True, filename_as_id=True, required_exts=[".pdf", ".docx", ".xlsx", ".pptx"])

documents = []
for docs in reader.iter_data():
    file_filename = [x for x in glob.glob("".join([user_path,"MedEd-AI/Data/coursedata_*/course_file.json"]),recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
    file_df = pd.read_json(file_filename)
    file_metadata = file_df.loc[file_df['filename'] == docs[0].metadata['file_name'][7:].replace('_','+')]
    if file_metadata.empty != True:                
        file_metadata = file_metadata.squeeze().to_dict()
        file_metadata = pd.DataFrame(file_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
        file_metadata = file_metadata.to_dict('records')[0]
        folder = file_metadata.get('folder_id')
    else:
        file_metadata = {}
        folder = ''

    folder_filename = [x for x in glob.glob("".join([user_path,"MedEd-AI/Data/coursedata_*/course_folder.json"]),recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
    folder_df = pd.read_json(folder_filename)
    folder_metadata = folder_df.loc[folder_df['id'] == folder]
    if folder_metadata.empty != True:
        folder_metadata = folder_metadata.squeeze().to_dict()
        folder_metadata = pd.DataFrame(folder_metadata, index=[0]).replace(np.NaN, 0).replace(0, None)
        folder_metadata = folder_metadata.to_dict('records')[0]
        if 'Week' in folder_metadata['full_name']:
            week = [i for i in folder_metadata['full_name'].split("/") if 'Week' in i][0].replace('Week','').replace(' ','')
            folder_metadata.update({"week":week})
        full_name = folder_metadata['full_name'].split("/")[-1]
        folder_metadata.update({"folder_name":full_name})
    else:
        folder_metadata = {}
    
    course_filename = [x for x in glob.glob("".join([user_path,"MedEd-AI/Data/coursedata_*/course_course.json"]),recursive=True) if docs[0].metadata['file_name'][0:6] in x][0]
    course_df = pd.read_json(course_filename)        
    course_metadata = course_df.loc[course_df['id'] == folder_metadata.get('context_id')]
    if course_metadata.empty != True:
        course_metadata = course_metadata.squeeze().to_dict()
        course_id = folder_metadata.get('context_id')
    else:
        course_metadata = {}
            
    for doc in docs:
        doc.metadata.update({"file_id": file_metadata.get('id'), "folder_id":file_metadata.get('folder_id'), "display_name":file_metadata.get('display_name')})
        doc.metadata.update({"week": folder_metadata.get('week'),  "folder_name": folder_metadata.get('folder_name')})
        doc.metadata.update({"course_id": course_metadata.get('id'), "course_name":course_metadata.get('name'),"course_code":course_metadata.get('course_code'),"course_term":course_metadata.get('term', {}).get('name')}) 
    documents.extend(docs)


parser = LangchainNodeParser(RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
))

nodes = parser.get_nodes_from_documents(documents)

vector_store = DuckDBVectorStore(nodes=nodes)
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection, nodes=nodes)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

vector_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model, llm=llm
)



**********
Trace: index_construction
    |_node_parsing -> 0.431932 seconds
      |_chunking -> 0.001126 seconds
      |_chunking -> 0.000747 seconds
      |_chunking -> 0.000605 seconds
      |_chunking -> 0.000565 seconds
      |_chunking -> 0.000398 seconds
      |_chunking -> 0.000206 seconds
      |_chunking -> 0.000158 seconds
      |_chunking -> 0.000466 seconds
      |_chunking -> 0.000268 seconds
      |_chunking -> 0.000794 seconds
      |_chunking -> 0.000627 seconds
      |_chunking -> 0.000499 seconds
      |_chunking -> 0.00054 seconds
      |_chunking -> 0.000363 seconds
      |_chunking -> 0.000165 seconds
      |_chunking -> 0.00038 seconds
      |_chunking -> 0.000227 seconds
      |_chunking -> 0.000678 seconds
      |_chunking -> 0.000419 seconds
      |_chunking -> 0.000391 seconds
      |_chunking -> 0.000367 seconds
      |_chunking -> 0.000296 seconds
      |_chunking -> 0.000315 seconds
      |_chunking -> 0.000663 seconds
      |_chunking -> 0.000415 seconds
 

### Check Metadata

In [ ]:
nodes[0]

### Define RetrieverQueryEngine

In [13]:
vector_store_info = VectorStoreInfo(
    content_info="Student guides to help prepare for consolidation assessments",
    metadata_info=[
        MetadataInfo(
            name="page_label",
            description="What page of the file the context is from",
            type="string",
        ),
        MetadataInfo(
            name="file_name",
            description="The name of the file the context is from",
            type="string",
        ),
        MetadataInfo(
            name="file_path",
            description="The file path of the context file",
            type="string",
        ),
        MetadataInfo(
            name="file_type",
            description="The type of file",
            type="string",
        ),
        MetadataInfo(
            name="file_size",
            description="The size of the file in bytes",
            type="integer",
        ),
        MetadataInfo(
            name="creation_date",
            description="When the file was created",
            type="string",
        ),
        MetadataInfo(
            name="last_modified_date",
            description="When the file was last modified",
            type="string",
        ),
        MetadataInfo(
            name="display_name",
            description="The name of the file",
            type="string",
        ),
        MetadataInfo(
            name="week",
            description="The week the context was administered",
            type="string",
        ),
        MetadataInfo(
            name="folder_name",
            description="The course folder that contains the file",
            type="string",
        ),
        MetadataInfo(
            name="course_id",
            description="The unique identifier of the course",
            type="integer",
        ),
        MetadataInfo(
            name="course_name",
            description="The full name of the course",
            type="string",
        ),
        MetadataInfo(
            name="course_code",
            description="The shortened name of the course",
            type="string",
        ),
        MetadataInfo(
            name="course_term",
            description="What term the course was offered in",
            type="string",
        ),
    ],
)

vector_auto_retriever = VectorIndexAutoRetriever(
    vector_index, vector_store_info=vector_store_info
)

retriever_query_engine = RetrieverQueryEngine.from_args(
    vector_auto_retriever, llm=llm
)

### Set Up Query Engine Tools

In [14]:
from llama_index.core.tools import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over"
        " a table svr_course_quiz_view, containing columns:"
        " quiz_id, quiz_type, quiz_title, history_id, submission_id,"
        " student_score, quiz_question_count, quiz_points_possible, question_points_possible," 
        "answer_points_scored, attempt, question_name, question_type, question_text, question_answer, and student_answer"
    ),
)


vector_tool = QueryEngineTool.from_defaults(
        query_engine=retriever_query_engine,
        description=f"Useful for answering semantic questions about consolidation assessments, and general course-related questions like when certain material is being taught",
    )


### Define Query Engine

In [15]:
query_engine = SQLAutoVectorQueryEngine(
    sql_tool, 
    vector_tool,
    llm=llm
)

### Query Test 1: Ability to identify topics covered in a specific assessment

In [16]:
response = query_engine.query("What material is covered in CBB 3 Consolidation Assessment Exercise?")

response.response

Querying other query engine: The question is asking about the material covered in a specific assessment, which is related to consolidation assessments and general course-related questions.
Query Engine response: The material covered in the CBB 3 Consolidation Assessment Exercise includes proteolysis and protein degradation, protease families and mechanism, comparing proteolytic processes, protein degradation rates, lysosomal proteolysis and autophagy, overview of ubiquitination, ubiquitin conjugation machinery, how E3s work and how they are regulated, and how the proteasome degrades ubiquitinated proteins.
**********
Trace: query
**********


'The material covered in the CBB 3 Consolidation Assessment Exercise includes proteolysis and protein degradation, protease families and mechanism, comparing proteolytic processes, protein degradation rates, lysosomal proteolysis and autophagy, overview of ubiquitination, ubiquitin conjugation machinery, how E3s work and how they are regulated, and how the proteasome degrades ubiquitinated proteins.'

### Query Test 2: Ability to query SQL db, synthesize results and deliver a cohesive answer

In [22]:
response = query_engine.query("What course is offered in fall 2023?")

response.response += ' '
out = response.response.split('. ')[:-1]
print('.\n'.join(out))

print(response)

Querying other query engine: The second choice is more relevant because it mentions answering course-related questions and knowing when certain material is being taught.
Query Engine response: PWY 100: Foundations - HMS - 08/07/2023 - 11/03/2023
**********
Trace: query
**********

PWY 100: Foundations - HMS - 08/07/2023 - 11/03/2023 


### Query Test 3: Ability to identify material from specific time, create quiz questions, create Google quiz

In [ ]:
response = query_engine.query(
    'Identify what material is taught in week 7. Then create a 5 question review quiz on the material taught in week 7. Do not ask questions about text from a photo/graph. Do not ask any questions about textbook sections. Do not ask any questions about learning objectives. Every question should have only one correct answer. Every question should have 4 potential answers. Format the output like this:{"info":{"title": "", "description": "", "questions": [ { "question": "","type": "RADIO","options": [], "correct_answer": ""}]}}'
)

# Google Quiz Creation
form = Form(file_type='credentials', loginfile=CREDENTIALS_FILE,
        discovery_doc=DISCOVERY_DOC, scopes=SCOPES, sentence_response = response)


### Chat Session Testing